In [1]:
import deimos_router

In [15]:
from deimos_router import Router, register_router, chat
from deimos_router.rules import TaskRule, AutoTaskRule, CodeRule

## Basic Usage - Task Based Routing

A Router uses one or more Rules to select a model. The simplest Rule is TaskRule, which maps task names to models.

In [3]:
Router(
   name="my-first-router",
   rules=[
       TaskRule(
           name="task-based-routing",
           triggers={
               'coding': 'openai/gpt-5',
               'creative': 'openai/gpt-4o',
               'simple': 'openai/gpt-5-nano'
           }
       )
   ],
   default="openai/gpt-4o-mini"
)



To use a Router, make a call to `chat.completions.create`, as if it were a call to OpenAI's SDK, and specify the router as the model.

For simple task-based routing, include the task name as an argument.

In [4]:
# Use the router for chat completions
response = chat.completions.create(
   model="deimos/my-first-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response.choices[0].message.content)

To find the nth Fibonacci number, you can use an iterative approach, a recursive approach, or employ dynamic programming. The iterative approach is efficient and widely used due to its simplicity and low overhead. Below is an implementation of this approach:

```python
def fibonacci(n):
    if n <= 0:
        raise ValueError("n must be a positive integer")
    elif n == 1:
        return 0
    elif n == 2:
        return 1

    prev, curr = 0, 1
    for _ in range(3, n + 1):
        prev, curr = curr, prev + curr

    return curr

# Example usage:
n = 10
print(f"The {n}th Fibonacci number is: {fibonacci(n)}")
```

This function `fibonacci` calculates the nth Fibonacci number starting from F(1) = 0 and F(2) = 1. It handles edge cases when n is less than or equal to zero, and it iteratively computes Fibonacci numbers by maintaining the last two numbers in the sequence and updating them as it progresses to the desired position. When calling the function, the nth Fibonacci number is retur

Details about the routing decision can be found in the response at `._deimos_metadata`.

In [5]:
response._deimos_metadata

{'router_used': 'my-first-router',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'available_models': [],
 'explain': [{'rule_type': 'TaskRule',
   'rule_name': 'task-based-routing',
   'rule_trigger': 'coding',
   'decision': 'openai/gpt-4o'}]}

## AutoTask Routing

An `AutoTaskRule` is created in the same way as a `TaskRule`, but the task is determined by a call to a small language model.

In [6]:
AutoTaskRule(
    name = "auto-task-rule",
    triggers = {
        "creative writing" : "openai/gpt-4o",
        "writing code" : "openai/gpt-5",
        "informational" : "openai/gpt-5-mini",
        "haiku composition" : "openai/gpt-5-nano"
    },
)


        

In [7]:
auto_router = Router(
    name = "auto-router",
    rules = [auto_task],
    default="openai/gpt-4o-mini"
    )


In [8]:
# Use the router for chat completions
response1 = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response1.choices[0].message.content)

def fib(n: int) -> int:
    """
    Return the nth Fibonacci number with F(0)=0, F(1)=1.

    Uses fast doubling in O(log n) time.
    Raises:
        ValueError: if n is negative.
    """
    if n < 0:
        raise ValueError("n must be non-negative")

    def _dob(k: int) -> tuple[int, int]:
        # Returns (F(k), F(k+1))
        if k == 0:
            return 0, 1
        a, b = _dob(k >> 1)  # a=F(m), b=F(m+1) for m=k//2
        c = a * (2 * b - a)  # F(2m)
        d = a * a + b * b    # F(2m+1)
        if k & 1:
            return d, c + d  # (F(2m+1), F(2m+2))
        else:
            return c, d      # (F(2m), F(2m+1))

    return _dob(n)[0]


In [9]:
response1._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'writing code',
   'decision': 'openai/gpt-5'}]}

In [10]:
haiku = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a short japanese poem about mars, and follow the syllable count 5, 7, 5"}
   ],
)

print(haiku.choices[0].message.content)

かせいよる
ひかりはあかく
またあえる


In [11]:
haiku._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'haiku composition',
   'decision': 'openai/gpt-5-nano'}]}

# Code / Not code Routing

The `CodeRule` determines whether a prompt contains code and routes based on that.

In [16]:
code_or_not = CodeRule(
    name = "code-or-not-code",
    code = "openai/gpt5",
    not_code = "openai/gpt-4o"
)

Router(
    name = "code-nocode-router",
    rules = [code_or_not]
)

Router(name='code-nocode-router', models=[])

In [17]:
prompt = """
Debug this:
```
def multiply(x, y):
    return x**y
```
"""


might_be_code_1 = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
   task="coding"
)

print(might_be_code_1.choices[0].message.content)

You used exponentiation (**) instead of multiplication (*).

Corrected code:
```python
def multiply(x, y):
    return x * y
```


In [18]:
might_be_code_1._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'writing code',
   'decision': 'openai/gpt-5'}]}